# Motivation

In my research, I have been using existing libraries to build neural network models. For example, [Keras](https://keras.io/) provides an extremely modular and clear Python API which sits on top of [Tensorflow](https://www.tensorflow.org/). It is very easy to train and experiment with neural networks, without needing to understand all the details under the hood. This was creating using the content of [Deep Learning Book](http://www.deeplearningbook.org/) chapter 6.

Sometimes it's good to understand all of the details. I would like to better understand the backpropagation algorithm. As such, I am implementing a Multi Layer Perception from scratch using only Python and Numpy.

# Theory

Neural networks can be thought of as sequence of tranformations made to input data.

$$y = f(x)$$

Deep neural networks make very many such transformation, which creates complex representations of the data.

$$y = f(f...f(f(x)))$$


Given examples of known $x$ and $y$, we want to define a mapping between them which reliably takes as input $x$ values and outputs values in $y$. Further, we want to capture some underlying structure that relates $x$ and $y$, meaning if another example $x^*$ are sampled in the same way as $x$, and the model has not seen it during training, we can predict ahead of time what the corresponding $y^*$ would be.

What form does this tranformation $f$ take?

We take:

$$ f = g(0, w^Tx + c) $$

where $w$ and $c$ are vectors. $w^Tx + c$ constitutes what is known as an [affine map](https://en.wikipedia.org/wiki/Affine_transformation). Affine maps are transformations that preserve points, lines and planes. 

g(z) = $max(0,z)$ is a known as the [Relu](https://en.wikipedia.org/wiki/Rectifier_(neural_networks) operator. This simple non-linear operation allows the series of functions $y = f(f...f(f(x)))$ to become non-linear. Note that if each $f$ only comprised the affine map $w^Tx + c$, then the resulting composition $y = f(f...f(f(x)))$ would always be linear.

Note this this is similar to the formulation of logistic regression. In logistic regression we find vectors $w$ and $c$ such that:

$$y = \sigma(w^T x + c)$$

minimises the cross-entropy of the true and predicted distributions. Minimising this cross entropy defines the cost function. Cross entropy is defined as:

$$H(p, q) = - \sum_i p_i(x) \log q_i(x) = \mathop{{}\mathbb{E}}[-\log q] = H(p) + D_{KL}(p\|q) $$ 

where $H(p) = -\sum_i p_i\log p_i$ is the [entropy](https://en.wikipedia.org/wiki/Entropy_(information_theory) of $p$ and $D_{KL}(p\|q) = \sum_i p_i\log \frac{p_{i}}{q_{i}}$ is the [Kullback-Leibler divergence](https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence) of $p$ given $q$.

In the case of our real values $y$ and our predicted values $y'$. Since we are predicting probabilities, $p \in \{y, 1-y\}$ and $q \in \{y', 1-y'\}$ the cross-entropy loss function is defined by:

$$L(y,y') = - \sum_i p_i(x) \log q_i(x) = -\bigl(y \log y' + (1-y) \log (1-y')\bigr)$$ 

# The Chain Rule

In calculus, the chain rule is used to define the derivate of composite functions. If $y = g(x)$ and $z = f(g(x)) = f(y)$, then the chain rule states that:

$$ \frac{dz}{dx} = \frac{df(g(x))}{dx} = \frac{df(y)}{dx} =  \frac{df}{dy}\frac{dy}{dx} = \frac{dz}{dy}\frac{dy}{dx} $$


This generalizes beyond the scalar case. Suppose that $x \in \mathbb{R}^m$, $y \in \mathbb{R}^n$. Let $g$ be a mapping from $\mathbb{R}^m$ to $\mathbb{R}^n$, and let $f$ map from $\mathbb{R}^n$ to $\mathbb{R}$

Then,

$$ \frac{\partial z}{\partial x_i} = \sum_{j}\frac{\partial z}{\partial y_j}\frac{\partial y_j}{\partial x_i}$$

In vector notation this is represented by:

$$\nabla_\mathbf{x} z = \Bigl( \frac{\partial \mathbf{y}}{\partial \mathbf{x}} \Bigr)^T \nabla_{\mathbf{x}} z$$

where $\frac{\partial \mathbf{y}}{\partial \mathbf{x}}$ is the $n \times m$ Jacobian matrix of $g$. We read this as the gradient of $z$ with respect to the vector $\mathbf{x}$.


We see from this that we obtain the gradient of a variable $\mathbf{x}$ by multiplying a Jacobian matrix $\frac{\partial \mathbf{y}}{\partial \mathbf{x}}$ by a gradient $\Delta_{\mathbf{x}} z$

In the context of deep learning we do not just operate on vectors, but also on tensors. We can compute the gradient of $z$ with respect to a tensor $X$ as:

$$\nabla_\mathbf{x} z = \sum_j (\nabla_{\mathbf{X}} Y_j) \frac{\partial z}{\partial Y_j}$$

Normally, $(\nabla_{\mathbf{X}} z)_i$ corresponds to $\frac{\partial z}{\partial X_i}$. For a vector, this enumerates all possible elements in the vector. For tensors, we just flatten across all the axis to achieve one long vector.

# Forward propagation

We can now describe the propagation algorithm

**Algorithm**

$l$ Network depth. <br>
$\mathbf{W}^{(i)}, i \in \{1,...l\}$ weight parameters <br>
$\mathbf{b}^{(i)}, i \in \{1,...l\}$ bias parameters <br>
$\mathbf{x}$ input
$\mathbf{y}$ target output <br>

* $\mathbf{h}^{(0)}$ = $x$
* **for** $k = 1, ..., l$ **do**
    * $\mathbf{a}^{(k)} = \mathbf{b}^{(k)} + \mathbf{W}^{(k)}h^{(k)}$
    * $\mathbf{h}^{(k)} = f(\mathbf{a}^{(k)})$
* **end for**
* $\mathbf{y'} = \mathbf{h}^{(l)}$
* J = $L(y',y) + \lambda \Omega(\theta)$
    


We will implement a Multi Layer Perception with a single hidden layer. As such, $\mathbf{W}$ and $\mathbf{b}$ have two columns. The number of rows is defined by the number of hidden units, $h$ of the model.

* $a_1 = W_1X + b_1 $
* $h_1 = relu(a_1)$
* $a_2 = W_2h_1 + b_2$
* $y' = \sigma(a_2)$
* $e = L(y',y)$

A forward pass on our data will take a matrix $X$ and produce a vector of predictions $Y'$. We defined a cost function based on the cross-entropy of $Y'$ with respect to $Y$, defined above. This gives us $e$ = $L(Y,Y') = -(Y_i \log Y_i' + (1-Y_i) \log (1-Y_i'))$. Note that $e$ is a vector in its current form.

We would like to know what the gradient of $e$ is with respect to all of our parameters, and then update them by a small amount to minimise the cost.

The algorithm for backpropagation from the Deep Learning book is terse.

* $\mathbf{g}$ = $\nabla_{y'}J$ Compute gradient on final layer.
* **for** $k = l, ..., 1$
    * $\mathbf{g} = \nabla_{\mathbf{a}^{(k)}}J = \mathbf{g} \odot f^{'}(\mathbf{a}^{(k)})$ Gradient on layer output. Element-wise multiplication if $f$ is element-wise.
    * $\nabla_{\mathbf{b}^{(k)}}J = \mathbf{g} + \lambda \nabla_{\mathbf{b}^{(k)}}\Omega(\theta)$ Compute gradients on biases
    * $\nabla_{\mathbf{W}^{(k)}}J = \mathbf{g}\ \mathbf{h}^{(k-1)\top} + \lambda \nabla_{\mathbf{W}^{(k)}}\Omega(\theta)$ Compute gradients on weights
    * $ \mathbf{g} = \nabla_{\mathbf{h}^{(k-1)}}J = \mathbf{W}^{(k)\top}$ Propagate gradients w.r.t lower-level hidden layer activations.
* **end for**

Here is a function diagram displaying our operations and intermediate values.

$\require{AMScd}$
\begin{CD}
    X @>W_1, b_1>> a_1 @>relu>> h_1 @>W_2, h_2>> a_2 @>\sigma>> Y' @>L>> e
\end{CD}

That is, we would like expressions for: $\frac{\partial e}{\partial W_2}, \frac{\partial e}{\partial b_2}, \frac{\partial e}{\partial W_1}, \frac{\partial e}{\partial b_1}$


Using the chain rule, we can see that:
$$ \frac{\partial e}{\partial W_2} = \frac{\partial e}{\partial a_2}\frac{\partial a_2}{\partial W_2} , \ \ \
\frac{\partial e}{\partial a_2} = \frac{\partial e}{\partial Y'}\frac{\partial Y'}{\partial a_2}  $$

So we see that,

$$\frac{\partial e}{\partial W_2} = \frac{\partial e}{\partial Y'}\frac{\partial Y'}{\partial a_2}\frac{\partial a_2}{\partial W_2}, \ \ \ \frac{\partial e}{\partial b_2} = \frac{\partial e}{\partial Y'}\frac{\partial Y'}{\partial a_2}\frac{\partial a_2}{\partial b_2}$$

Therefore, we need to find expressions for:
$$\frac{\partial a_2}{\partial b_2}, \frac{\partial a_2}{\partial W_2}, \frac{\partial Y'}{\partial a_2},   \frac{\partial e}{\partial Y'} $$

Now, $$e = -\bigl(Y \log Y' + (1-Y) \log (1-Y')\bigr) \implies \frac{\partial e}{\partial Y'} = -\Bigl(\frac{Y}{Y'} - \frac{(1-Y)}{(1-Y')}\Bigr)$$ 

$$$$

Also, $$ Y' = \sigma (a_2) \implies \frac{\partial y'}{\partial a_2} = \sigma(a_2)(1 - \sigma(a_2))$$

Substituting $Y_{i}' = \sigma(a_{i2})$

$$ \frac{\partial e}{\partial a_2} = \Bigl(\frac{Y}{\sigma(a_{2})} - \frac{(1-Y)}{(1-\sigma(a_{2}))}\Bigr) \sigma(a_{2})(1 - \sigma(a_{2})) 
= \Bigl((1 - \sigma(a_{2}))Y - \sigma(a_{2})(1-Y)\Bigr) = (Y' - Y)$$

$\frac{\partial e}{\partial a_2} = (Y' - Y)$ is a remarkably simple expression, given the complexities of our loss function $L$, and final activation function $\sigma$. In fact, it has been chosen to be so simple.

Let $\frac{\partial e}{\partial a_2} = \delta_3$. Let us first calculate $\frac{\partial e}{\partial a_1}$ and name it $\delta_2$

$$\frac{\partial e}{\partial a_1} = \frac{\partial e}{\partial a_2}\frac{\partial a_2}{\partial a_1} = \delta_3 \frac{\partial a_2}{\partial h_1} \frac{\partial h_1}{\partial a_1} = \delta_3 W_{2}^T \frac{\partial h_1}{\partial a_1}$$

Since $h_1 = relu(a_1), \frac{\partial h_1}{\partial a_1}$ is $1 $ for $a_1 > 0$ and $0$ otherwise. Call this function $drelu(a_1)$. Therefore: 

$$\delta_2 = \delta_3 W_{2}^T \circ drelu(a_1)$$

We can now write down expressions for $\frac{\partial e}{\partial W_2}, \frac{\partial e}{\partial b_2}, \frac{\partial e}{\partial W_1}, \frac{\partial e}{\partial b_1}$

$$\frac{\partial e}{\partial W_2}  = \frac{\partial e}{\partial a_2} \frac{\partial a_2}{\partial W_2} = h_1^T\delta_3$$

and,

$$\frac{\partial e}{\partial b_2}  = \frac{\partial e}{\partial a_2} \frac{\partial a_2}{\partial b_2} = \delta_3$$

Also,

$$\frac{\partial e}{\partial W_1} = \frac{\partial e}{\partial a_2}\frac{\partial a_2}{\partial a_1}\frac{\partial a_1}{\partial W_1} = X^T \circ \delta_2$$

and

$$\frac{\partial e}{\partial b_1} = \frac{\partial e}{\partial a_2}\frac{\partial a_2}{\partial a_1}\frac{\partial a_1}{\partial b_1} = \delta_2$$ 

# Implementation

We will use the breast cancer data dataset. This defines a set of 30 predicters that are used to classify malignant vs benign breast cancer tumours.

In [494]:
import scipy
import sklearn.datasets
cancer_data = sklearn.datasets.load_breast_cancer()
Y = cancer_data.target
X = cancer_data.data
X = (X - np.mean(X,axis=0)) / np.std(X,axis=0)
print (X.shape, y.shape)

((569, 30), (569, 1))


We randomly initialise the weight and bias parameters.

In [770]:
import numpy as np

def rand_init(s,h):
    return (np.random.random(size=(s,h)) - 0.5)/10


We compute the forward pass where the network transforms the input $X$ input output $y$. We convert the final output of the neural network to a binary prediction using the $\sigma$ function.

$$ \sigma(x) = \frac{1}{1 + \exp(-x)} $$

In [850]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))
W = {1: rand_init(30,5), 2: rand_init(5,1)}
h = {}
b = {1: rand_init(1,5), 2: rand_init(1,1)}
a = {}
def forward_pass(x):

    h[0] = X
    for k in [1,2]:
        a[k] = b[k] + np.dot(h[k-1],W[k])
        h[k] = a[k].copy()
        h[k][h[k] < 0] = 0
    y_pred = h[2]
    y_pred = sigmoid(y_pred)
    return y_pred

# Backward pass 

In addition to the input $\mathbf{x}$, we need a target $\mathbf{y}$. This computation yields the gradients on the activations $\mathbf{a} ^{(k)}$ for each layer $k$. From the gradients on each layer, we can find the gradients on the parameters in each layer and incorporate them into a stochastic gradient descent update.

We also need to define the cost function, $J$, as the cross-entropy between $y$ and $y'$.

**Algorithm**

In [797]:
def cost(y_real,y_pred):
    return - y_real*np.log(y_pred) - (1-y_real)*np.log(1-y_pred)
#     return np.square(y_real-y_pred)

The derivate of the $\sigma$ function is:

$$ \frac{d \sigma}{dx} = \sigma(x)(1 - \sigma(x))$$

In [798]:
def dsigmoid(x):
    return sigmoid(x)*(1 - sigmoid(x))

The derivative of the Relu function is: $0$ for $x <= 0$, and $1$ for $x > 0$

In [799]:
def drelu(x):
    return 1 if x > 0 else 0
drelu = np.vectorize(drelu)

In [950]:
def backward_pass(y_pred):   
    delta_3 = (y-y_pred) # Compute delta_3 = de/da_2
    
    #First layer updates
    
    db_2 = np.sum(delta_3, axis=0, keepdims=True)
    dW_2 = h[1].T.dot(delta_3)
    
    delta_2 = np.multiply(delta_3.dot(W[2].T), drelu(a[1]))
    dW_1 = X.T.dot(delta_2)
    db_1 = np.sum(delta_2, axis=0, keepdims=True)

    W[1] -= learning_rate * dW_1
    b[1] -= learning_rate * db_1

    W[2] -= learning_rate * dW_2
    b[2] -= learning_rate * db_2

In [952]:
learning_rate = 0.00001
W = {1: rand_init(30,5), 2: rand_init(5,1)}
h = {}
b = {1: rand_init(1,5), 2: rand_init(1,1)}
a = {}


In [953]:
def train():
    for e in range(10):
        y_pred = forward_pass(X)
        backward_pass(y_pred)
        print (sum(cost(Y, predict(X))))        
def predict(X):
    predictions = []
    for i in range(X.shape[0]):
        predictions.append(forward_pass(X[i,:])[0][0])
    return np.array(predictions)

train()

393.427699022
393.204369971
392.982251122
392.761322332
392.541560437
392.322942418
392.105431123
391.888998762
391.673601197
391.459217244
